In [ ]:
# Install & Imports
!pip install sentence-transformers

import re
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [48]:
# Load Sentence-BERT model
sbert_model = SentenceTransformer("all-MiniLM-L6-v2")
print("✅ SBERT model loaded")

✅ SBERT model loaded


In [50]:
# Load Dataset
df = pd.read_csv("Dummy_Lawyers.csv")

def create_summary(row):
    return (
        f"{row['Name']} is a lawyer specializing in {row['Domain']} with "
        f"{row['Years of active experience']} years of experience. "
        f"Charges approximately ₹{row['Price']:.0f} and has a client satisfaction rating of "
        f"{row['Client satisfaction (out of 10)']}/10."
    )

df["summary_text"] = df.apply(create_summary, axis=1)

print("✅ Dataset loaded with", len(df), "lawyers")

✅ Dataset loaded with 1000 lawyers


In [51]:
# Precompute Embeddings
print("⏳ Generating SBERT embeddings...")
df["embedding"] = df["summary_text"].apply(lambda x: sbert_model.encode(x))
print("✅ Embeddings generated")

⏳ Generating SBERT embeddings...
✅ Embeddings generated


In [52]:
def recommend(query):
    filtered_df = df.copy()
    query_lower = query.lower()

    # --- Experience filter ---
    exp_match = re.search(r"(less|more) than (\d+)\s*years?", query_lower)
    if exp_match:
        op, val = exp_match.groups()
        val = int(val)
        if op == "less":
            filtered_df = filtered_df[filtered_df["Years of active experience"] < val]
        else:
            filtered_df = filtered_df[filtered_df["Years of active experience"] > val]

    # --- Budget filter ---
    budget_match = re.search(r"(?:budget of|under|below)\s*₹?(\d+)", query_lower)
    if budget_match:
        budget = int(budget_match.group(1))
        filtered_df = filtered_df[filtered_df["Price"] <= budget]

    # --- Domain filter ---
    for domain in df["Domain"].unique():
        if domain.lower() in query_lower:
            filtered_df = filtered_df[filtered_df["Domain"].str.lower() == domain.lower()]
            break

    if filtered_df.empty:
        print("⚠️ No lawyers match your query after filtering.")
        return

    # --- Embedding similarity ---
    query_embedding = sbert_model.encode(query)
    lawyer_embeddings = list(filtered_df["embedding"])
    sim_scores = cosine_similarity([query_embedding], lawyer_embeddings)[0]

    filtered_df["Similarity Score"] = sim_scores
    filtered_df = filtered_df.sort_values(by="Similarity Score", ascending=False)

    result = filtered_df[[
        "Name", "Domain", "Years of active experience", "Price",
        "Client satisfaction (out of 10)", "Similarity Score"
    ]].rename(columns={
        "Years of active experience": "Experience (yrs)",
        "Price": "Estimated Price (₹)",
        "Client satisfaction (out of 10)": "Satisfaction (/10)"
    })

    print(f"\n📋 Lawyers matching query: \"{query}\"")
    print(result.to_string(index=False))
    return result

In [53]:
recommend("give me names of lawyers who have less than 15 years of experience in civil law for budget of 20000rs")
recommend("criminal law lawyer with more than 10 years of experience under 50000")
recommend("family law expert under 25000 with at least 5 years experience")
recommend("property lawyer more than 20 years of experience")


📋 Lawyers matching query: "give me names of lawyers who have less than 15 years of experience in civil law for budget of 20000rs"
                Name    Domain  Experience (yrs)  Estimated Price (₹)  Satisfaction (/10)  Similarity Score
        William York Civil Law                 7              6617.51                 5.0          0.581037
         Lori Martin Civil Law                 2             15520.36                 7.1          0.533213
    Patricia Cameron Civil Law                 8              8718.26                 5.4          0.527954
          Gary Young Civil Law                 1             10730.61                 6.4          0.523268
        James Arnold Civil Law                 2             17657.24                 6.3          0.521212
       Garrett Boone Civil Law                 2             13627.33                 5.2          0.513520
         David Lyons Civil Law                 6             12248.81                 5.5          0.503380
     

,Name,Domain,Experience (yrs),Estimated Price (₹),Satisfaction (/10),Similarity Score
552,Jamie Davidson,Property Law,31,18892.97,8.0,0.529663
922,William Lee,Property Law,36,30663.67,8.5,0.526471
143,Timothy Young,Property Law,23,11415.93,5.4,0.520035
192,Jeffrey Richardson,Property Law,21,15749.45,6.4,0.497019
997,Angela Long,Property Law,40,21428.61,7.0,0.491241
645,Kenneth Garcia,Property Law,36,21862.61,7.7,0.490696
46,Ellen Curry,Property Law,25,16645.03,7.8,0.484486
815,Angela Griffin,Property Law,22,12090.83,4.8,0.478756
665,Steven Tucker,Property Law,39,24217.02,8.9,0.477568
978,Jennifer Wolf,Property Law,39,26361.27,8.8,0.477247
